## 1.0 Initialization and Setup

1.1 Importing Libraries

In [1]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.34
Running PyMongo Version: 4.15.3


1.2 Setting up connections to MySQL and MongoDB

In [3]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Narat04!",
    "hostname" : "localhost",
    "dbname" : "adventureworks"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "Tarangupta",
    "password" : "ds2002",
    "cluster_name" : "cluster0",
    "cluster_subnet" : "2ypncz8",
    "cluster_location" : "atlas", # "local"
    "db_name" : "adventureworks"
}

In [4]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

1.3 Data Exploration and Extractoin

At this point, the provided MySQL Code was run to create the Adventureworks database, including setting up all of the proivded dimension tables stored as views. Furthermore, the dim_date table was created using the code from lab 02c. For the purposes of this project, the "*product*" and "*productinventory*" tables were exported to a .csv file and the "*product vendor*" and "*vendor*" tables were exported as json files and uploaded to MongoDB. The remaining tables (*"location"*, *"dim_date"*) will be accessed directly in MySQL. The code files that create the Adventureworks data and views are included in the repository. The necessary tables were exported directly from MySQL table viewer, so there is no code to show that, however all exported files are included in the repository as well.

**The goal of this analysis is to create an Inventory Fact Table, analyzing the product inventory, location, and vendor supply.**

*note: dim_date range was changed to 01-01-1995 to 12-31-2010 to ensure all adventureworks dates are included*

1.4 Uploading data tables to mongoDB

In [5]:
client = get_mongo_client(**mongodb_args)

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd())

json_files = {"vendor_table" : 'vendor_table.json',
              "vendorproduct_table" : 'vendorproduct_table.json',
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)   

## 2.0 Importing data tables into python

2.1 Importing tables from csv

In [6]:
# importing product csv file using pandas
df_product = pd.read_csv("product_table.csv")
df_product.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,NaN,NaN,...,2004-03-11 10:01:36
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,NaN,NaN,...,2004-03-11 10:01:36
2,3,BB Ball Bearing,BE-2349,1,0,NaN,800,600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,NaN,NaN,...,2004-03-11 10:01:36
3,4,Headset Ball Bearings,BE-2908,0,0,NaN,800,600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,NaN,NaN,...,2004-03-11 10:01:36
4,316,Blade,BL-2036,1,0,NaN,800,600,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,NaN,NaN,...,2004-03-11 10:01:36


In [7]:
# importing productinventory csv file using pandas
df_productinventory = pd.read_csv("productinventory_table.csv")
df_productinventory.head()

,ProductID,LocationID,Shelf,Bin,Quantity,rowguid,ModifiedDate
0,1,1,A,1,408,...,2004-09-08 00:00:00
1,1,6,B,5,324,...,2004-09-08 00:00:00
2,1,50,A,5,353,...,2004-09-08 00:00:00
3,2,1,A,2,427,...,2004-09-08 00:00:00
4,2,6,B,1,318,...,2004-09-08 00:00:00


2.2 Importing tables directly from MySQL

In [8]:
# importing dim_date table from MySQL using get_sql_dataframe function
sql_dim_date = "SELECT * FROM adventureworks.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.head()

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,19950101,1995-01-01,1995/01/01,01/01/1995,01/01/1995,1,Sunday,1,1,Weekend,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
1,19950102,1995-01-02,1995/01/02,01/02/1995,02/01/1995,2,Monday,2,2,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
2,19950103,1995-01-03,1995/01/03,01/03/1995,03/01/1995,3,Tuesday,3,3,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
3,19950104,1995-01-04,1995/01/04,01/04/1995,04/01/1995,4,Wednesday,4,4,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3
4,19950105,1995-01-05,1995/01/05,01/05/1995,05/01/1995,5,Thursday,5,5,Weekday,...,N,1,1995,1995-01,1995Q1,7,3,1995,1995-07,1995Q3


In [9]:
# importing location table from MySQL using get_sql_dataframe function
sql_location = "SELECT * FROM adventureworks.location;"
df_location = get_sql_dataframe(sql_location, **mysql_args)
df_location.head()

,LocationID,Name,CostRate,Availability,ModifiedDate
0,1,Tool Crib,0.0,0.0,1998-06-01
1,2,Sheet Metal Racks,0.0,0.0,1998-06-01
2,3,Paint Shop,0.0,0.0,1998-06-01
3,4,Paint Storage,0.0,0.0,1998-06-01
4,5,Metal Storage,0.0,0.0,1998-06-01


2.3 Importing tables from MongoDB

In [10]:
# importing vendor table from MongoDB using get_mongo_dataframe function
mongo_client = get_mongo_client(**mongodb_args)
df_vendor = get_mongo_dataframe(mongo_client, mongodb_args['db_name'], 'vendor_table', {})
df_vendor.head()

,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,PurchasingWebServiceURL,ModifiedDate
0,1,INTERNAT0001,International,1,1,1,None,2002-02-25 00:00:00
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,1,1,None,2002-02-17 00:00:00
2,3,PREMIER0001,"Premier Sport, Inc.",1,1,1,None,2002-03-05 00:00:00
3,4,COMFORT0001,Comfort Road Bicycles,1,1,1,None,2002-01-24 00:00:00
4,5,METROSP0001,Metro Sport Equipment,1,1,1,None,2002-03-01 00:00:00


In [11]:
# importing vendorproduct table from MongoDB using get_mongo_dataframe function
mongo_client = get_mongo_client(**mongodb_args)
df_vendorproduct = get_mongo_dataframe(mongo_client, mongodb_args['db_name'], 'vendorproduct_table', {})
df_vendorproduct.head() 

,ProductID,VendorID,AverageLeadTime,StandardPrice,LastReceiptCost,LastReceiptDate,MinOrderQty,MaxOrderQty,OnOrderQty,UnitMeasureCode,ModifiedDate
0,1,83,17,47.87,50.2635,2001-09-29 00:00:00,1,5,3.0,CS,2001-09-29 00:00:00
1,2,57,19,39.92,41.9160,2001-09-29 00:00:00,1,5,3.0,CTN,2001-09-29 00:00:00
2,4,85,17,54.31,57.0255,2001-09-29 00:00:00,1,5,NaN,CTN,2001-09-29 00:00:00
3,317,50,19,28.17,29.5785,2001-09-29 00:00:00,100,1000,300.0,EA,2001-09-29 00:00:00
4,317,84,17,25.77,27.0585,2001-09-25 00:00:00,100,1000,NaN,EA,2001-09-25 00:00:00


## 3.0 Cleaning Data Tables

3.1 Cleaning product df

In [12]:
drop_cols = ['SellEndDate','DiscontinuedDate','rowguid','ModifiedDate']
df_product.drop(drop_cols, axis=1, inplace=True)                           
df_product.head(2)

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate
0,1,Adjustable Race,AR-5381,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00
1,2,Bearing Ball,BA-8327,0,0,NaN,1000,750,0.0,0.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00


3.2 Cleaning productinventory df

In [13]:
drop_cols = ['rowguid']
df_productinventory.drop(drop_cols, axis=1, inplace=True)
df_productinventory.head(2)

,ProductID,LocationID,Shelf,Bin,Quantity,ModifiedDate
0,1,1,A,1,408,2004-09-08 00:00:00
1,1,6,B,5,324,2004-09-08 00:00:00


3.3 Cleaning vendor df

In [14]:
drop_cols = ['PurchasingWebServiceURL']
df_vendor.drop(drop_cols, axis=1, inplace=True)
df_vendor.head(2)

,VendorID,AccountNumber,Name,CreditRating,PreferredVendorStatus,ActiveFlag,ModifiedDate
0,1,INTERNAT0001,International,1,1,1,2002-02-25 00:00:00
1,2,ELECTRON0002,Electronic Bike Repair & Supplies,1,1,1,2002-02-17 00:00:00


## 4.0 Creating a Fact Table

4.1 Creating the fact table by joining product, product inventory, and location

In [15]:
df_fact_productinventory = pd.merge(df_productinventory, df_product, on='ProductID', how='left')
df_fact_productinventory = pd.merge(df_fact_productinventory, df_location, on='LocationID', how='left')
df_fact_productinventory.head()

,ProductID,LocationID,Shelf,Bin,Quantity,ModifiedDate_x,Name_x,ProductNumber,MakeFlag,FinishedGoodsFlag,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,Name_y,CostRate,Availability,ModifiedDate_y
0,1,1,A,1,408,2004-09-08 00:00:00,Adjustable Race,AR-5381,0,0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,Tool Crib,0.00,0.0,1998-06-01
1,1,6,B,5,324,2004-09-08 00:00:00,Adjustable Race,AR-5381,0,0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,Miscellaneous Storage,0.00,0.0,1998-06-01
2,1,50,A,5,353,2004-09-08 00:00:00,Adjustable Race,AR-5381,0,0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,Subassembly,12.25,120.0,1998-06-01
3,2,1,A,2,427,2004-09-08 00:00:00,Bearing Ball,BA-8327,0,0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,Tool Crib,0.00,0.0,1998-06-01
4,2,6,B,1,318,2004-09-08 00:00:00,Bearing Ball,BA-8327,0,0,...,NaN,NaN,NaN,NaN,NaN,1998-06-01 00:00:00,Miscellaneous Storage,0.00,0.0,1998-06-01


4.2 Dropping all unneeded columns in the fact table (alternative: only combine a subset of the columns to begin with)

In [16]:
drop_cols = ['Name_x','ProductNumber','Color','ProductModelID','SellStartDate','Name_y','ReorderPoint','SafetyStockLevel',
             'Class','Style','ProductLine','DaysToManufacture','SizeUnitMeasureCode','WeightUnitMeasureCode','MakeFlag','FinishedGoodsFlag']
df_fact_productinventory.drop(drop_cols, axis=1, inplace=True)
df_fact_productinventory.head(5)

,ProductID,LocationID,Shelf,Bin,Quantity,ModifiedDate_x,StandardCost,ListPrice,Size,Weight,ProductSubcategoryID,CostRate,Availability,ModifiedDate_y
0,1,1,A,1,408,2004-09-08 00:00:00,0.0,0.0,NaN,NaN,NaN,0.00,0.0,1998-06-01
1,1,6,B,5,324,2004-09-08 00:00:00,0.0,0.0,NaN,NaN,NaN,0.00,0.0,1998-06-01
2,1,50,A,5,353,2004-09-08 00:00:00,0.0,0.0,NaN,NaN,NaN,12.25,120.0,1998-06-01
3,2,1,A,2,427,2004-09-08 00:00:00,0.0,0.0,NaN,NaN,NaN,0.00,0.0,1998-06-01
4,2,6,B,1,318,2004-09-08 00:00:00,0.0,0.0,NaN,NaN,NaN,0.00,0.0,1998-06-01


4.3 Adding Vendor, Vendor Product, and Date tables to the fact table

In [17]:
# combing vendor and vendorproduct tables
combined_vendor_df = pd.merge(df_vendorproduct, df_vendor, on='VendorID', how='left')
drop_cols = ['UnitMeasureCode','AccountNumber','Name','ModifiedDate_x','ModifiedDate_y']
combined_vendor_df.drop(drop_cols, axis=1, inplace=True)

# combing the vendor tables with the fact table
df_fact_productinventory = pd.merge(df_fact_productinventory, combined_vendor_df, on='ProductID', how='left')

# inserting a date key column for ModifiedDate_x and Last_ReceiptDate
df_modified_date = df_dim_date[['date_key','full_date']]
df_modified_date = df_modified_date.rename(columns={"date_key" : "last_modified_date_key", "full_date" : "last_modified_date"})

df_fact_productinventory.rename(columns={"ModifiedDate_x" : "last_modified_date"}, inplace=True)
df_fact_productinventory['last_modified_date'] = pd.to_datetime(df_fact_productinventory['last_modified_date']).dt.date
df_fact_productinventory = pd.merge(df_fact_productinventory, df_modified_date, on='last_modified_date', how='left')

df_receipt_date = df_dim_date[['date_key','full_date']]
df_receipt_date = df_receipt_date.rename(columns={"date_key" : "last_receipt_date_key", "full_date" : "last_receipt_date"})

df_fact_productinventory.rename(columns={"LastReceiptDate" : "last_receipt_date"}, inplace=True)
df_fact_productinventory['last_receipt_date'] = pd.to_datetime(df_fact_productinventory['last_receipt_date']).dt.date
df_fact_productinventory = pd.merge(df_fact_productinventory, df_receipt_date, on='last_receipt_date', how='left')

# dropping unneded columns
drop_cols = ['ModifiedDate_y','last_receipt_date','last_modified_date','CreditRating','ActiveFlag','StandardPrice', 'LastReceiptCost', 'StandardCost', 'ListPrice', ]
df_fact_productinventory.drop(drop_cols, axis=1, inplace=True)

pd.set_option('display.max_columns', None)
df_fact_productinventory.head()

,ProductID,LocationID,Shelf,Bin,Quantity,Size,Weight,ProductSubcategoryID,CostRate,Availability,VendorID,AverageLeadTime,MinOrderQty,MaxOrderQty,OnOrderQty,PreferredVendorStatus,last_modified_date_key,last_receipt_date_key
0,1,1,A,1,408,NaN,NaN,NaN,0.00,0.0,83.0,17.0,1.0,5.0,3.0,1,20040908,20010929.0
1,1,6,B,5,324,NaN,NaN,NaN,0.00,0.0,83.0,17.0,1.0,5.0,3.0,1,20040908,20010929.0
2,1,50,A,5,353,NaN,NaN,NaN,12.25,120.0,83.0,17.0,1.0,5.0,3.0,1,20040908,20010929.0
3,2,1,A,2,427,NaN,NaN,NaN,0.00,0.0,57.0,19.0,1.0,5.0,3.0,1,20040908,20010929.0
4,2,6,B,1,318,NaN,NaN,NaN,0.00,0.0,57.0,19.0,1.0,5.0,3.0,1,20040908,20010929.0


4.4 Creating a new fact table key and reordering the columns

In [18]:
# adding fact table key
df_fact_productinventory.insert(0, "fact_productinventory_key", range(1, len(df_fact_productinventory) + 1)) 

# reordering the columns
ordered_columns = ['fact_productinventory_key', 'ProductID', 'LocationID', 'VendorID', 'last_modified_date_key', 'last_receipt_date_key', 'ProductSubcategoryID',
                   'Shelf', 'Bin', 'Quantity',  'Size', 'Weight', 'CostRate', 'Availability',
                    'AverageLeadTime', 'PreferredVendorStatus', 'MinOrderQty', 'MaxOrderQty', 'OnOrderQty']
df_fact_productinventory = df_fact_productinventory[ordered_columns]

# renaming the keys
df_fact_productinventory.rename(columns={"VendorID" : "vendor_key",
                                        "ProductID" : "product_key",
                                        "LocationID" : "location_key",
                                        "ProductSubcategoryID" : "product_subcategory_key"}, inplace=True)

# typecasting vendor_key to integer
df_fact_productinventory['vendor_key'] = df_fact_productinventory['vendor_key'].astype('Int64')

# final look at the fact table
df_fact_productinventory.head()

,fact_productinventory_key,product_key,location_key,vendor_key,last_modified_date_key,last_receipt_date_key,product_subcategory_key,Shelf,Bin,Quantity,Size,Weight,CostRate,Availability,AverageLeadTime,PreferredVendorStatus,MinOrderQty,MaxOrderQty,OnOrderQty
0,1,1,1,83,20040908,20010929.0,NaN,A,1,408,NaN,NaN,0.00,0.0,17.0,1,1.0,5.0,3.0
1,2,1,6,83,20040908,20010929.0,NaN,B,5,324,NaN,NaN,0.00,0.0,17.0,1,1.0,5.0,3.0
2,3,1,50,83,20040908,20010929.0,NaN,A,5,353,NaN,NaN,12.25,120.0,17.0,1,1.0,5.0,3.0
3,4,2,1,57,20040908,20010929.0,NaN,A,2,427,NaN,NaN,0.00,0.0,19.0,1,1.0,5.0,3.0
4,5,2,6,57,20040908,20010929.0,NaN,B,1,318,NaN,NaN,0.00,0.0,19.0,1,1.0,5.0,3.0


## 5.0 Loading Fact Table and Testing Queries

5.1 Loading Fact Table into MySQL

In [19]:
dataframe = df_fact_productinventory
table_name = "fact_productinventory"
primary_key = "fact_productinventory_key"
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [20]:
# Verifying data load by querying the fact table
sql_fact_pos = "SELECT * FROM adventureworks.fact_productinventory;"
df_fact_pos = get_sql_dataframe(sql_fact_pos, **mysql_args)
df_fact_pos.head()

,fact_productinventory_key,product_key,location_key,vendor_key,last_modified_date_key,last_receipt_date_key,product_subcategory_key,Shelf,Bin,Quantity,Size,Weight,CostRate,Availability,AverageLeadTime,PreferredVendorStatus,MinOrderQty,MaxOrderQty,OnOrderQty
0,1,1,1,83.0,20040908,20010929.0,NaN,A,1,408,None,NaN,0.00,0.0,17.0,1,1.0,5.0,3.0
1,2,1,6,83.0,20040908,20010929.0,NaN,B,5,324,None,NaN,0.00,0.0,17.0,1,1.0,5.0,3.0
2,3,1,50,83.0,20040908,20010929.0,NaN,A,5,353,None,NaN,12.25,120.0,17.0,1,1.0,5.0,3.0
3,4,2,1,57.0,20040908,20010929.0,NaN,A,2,427,None,NaN,0.00,0.0,19.0,1,1.0,5.0,3.0
4,5,2,6,57.0,20040908,20010929.0,NaN,B,1,318,None,NaN,0.00,0.0,19.0,1,1.0,5.0,3.0


5.2 Query Testing

In [21]:
# Query Test
# Total inventory by location and product
# Use 3 tables: fact_productinventory, location, product
# Perform Aggregation: SUM on Quantity and GROUP BY location and product
sql_inventory_by_location = """
SELECT 
    fpi.location_key,
    fpi.product_key,
    loc.Name AS LocationName,
    p.Name AS ProductName,
    SUM(fpi.Quantity) AS TotalQuantity
FROM adventureworks.fact_productinventory AS fpi
JOIN adventureworks.location AS loc 
    ON fpi.location_key = loc.LocationID
JOIN adventureworks.product AS p 
    ON fpi.product_key = p.ProductID
GROUP BY 
    fpi.location_key, 
    fpi.product_key, 
    loc.Name, 
    p.Name
ORDER BY 
    fpi.location_key,
    fpi.product_key;"""


df_inventory_by_location = get_sql_dataframe(sql_inventory_by_location, **mysql_args)
df_inventory_by_location.head()

,location_key,product_key,LocationName,ProductName,TotalQuantity
0,1,1,Tool Crib,Adjustable Race,408.0
1,1,2,Tool Crib,Bearing Ball,427.0
2,1,3,Tool Crib,BB Ball Bearing,585.0
3,1,4,Tool Crib,Headset Ball Bearings,512.0
4,1,317,Tool Crib,LL Crankarm,566.0


In [22]:
df_inventory_by_location.tail()

,location_key,product_key,LocationName,ProductName,TotalQuantity
995,60,962,Final Assembly,"Touring-3000 Yellow, 50",107.0
996,60,963,Final Assembly,"Touring-3000 Yellow, 54",36.0
997,60,964,Final Assembly,"Touring-3000 Yellow, 58",116.0
998,60,965,Final Assembly,"Touring-3000 Yellow, 62",72.0
999,60,966,Final Assembly,"Touring-1000 Blue, 46",86.0
